In [1]:
from dotenv import load_dotenv
load_dotenv() # load GOOGLE_API_KEY from .env file

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        max_tokens=None,
    )

# OWL Strength
Notice, in the .ttl file, how we
- Defined `marriedTo` as symmetric property, so we didn't have
to define the relation both for Joe and Love. We only defined love -> marriedTo -> joe, the triplet joe -> marriedTo -> love can be automatically inferred.
- Defined `killed` and `killedBy` as inverse, so by defining joe -> killed -> love, relation love -> killedBy -> joe can be automatically inferred.


In [6]:
from langchain_community.graphs import RdfGraph
graph = RdfGraph(
    source_file="owl_graph.ttl",
    standard="owl",
)

graph.load_schema()
graph.get_schema

'In the following, each IRI is followed by the local name and optionally its description in parentheses. \nThe OWL graph supports the following node types:\n<http://xmlns.com/ex/0.1/Male> (Male, None), <http://xmlns.com/ex/0.1/Person> (Person, None), <http://xmlns.com/ex/0.1/Fictional> (Fictional, None), <http://xmlns.com/ex/0.1/Female> (Female, None), <http://www.w3.org/2002/07/owl#ObjectProperty> (ObjectProperty, None), <http://www.w3.org/2002/07/owl#SymmetricProperty> (SymmetricProperty, None)\nThe OWL graph supports the following object properties, i.e., relationships between objects:\n<http://xmlns.com/ex/0.1/marriedTo> (marriedTo, None), <http://xmlns.com/ex/0.1/killed> (killed, None), <http://xmlns.com/ex/0.1/killedBy> (killedBy, None)\nThe OWL graph supports the following data properties, i.e., relationships between objects and literals:\n\n'

In [8]:
from langchain.chains import GraphSparqlQAChain
chain = GraphSparqlQAChain.from_llm(
    llm, graph=graph, verbose=True, allow_dangerous_requests=True, return_direct=False
)


In [9]:
res = chain.invoke("Is Love married? hint: use select query.\nDON'T surround the query with backticks")
# res = chain.invoke("Is Love alive?"
#                    "\nDON'T surround the query with backticks")



> Entering new GraphSparqlQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
PREFIX ex: <http://xmlns.com/ex/0.1/>
SELECT ?spouse
WHERE {
    ex:Love ex:marriedTo ?spouse .
}
Full Context:
[(rdflib.term.URIRef('http://xmlns.com/ex/0.1/JoeGoldberg'),)]

> Finished chain.


In [10]:
res["result"]

'Yes, based on the information I have, Love is married to Joe Goldberg.'